In [ ]:
!pip install tweepy pandas numpy plotly tqdm

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import time
import glob
import json                          
import plotly.graph_objects as go    
import tqdm
from tqdm import trange
from tqdm import tqdm
import datetime as dt
import requests

In [ ]:
# the secure way of defining api keys
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# import os
# bearer_token = os.environ.get("BEARER_TOKEN")

# the bad way of doing things
# api_key = '<api_key>'  # add in your api key from your Twitter Developer Account here
# api_key_secret = '<api_key_secret>' # add in your api secret from Twitter Developer Account here
# bearer_token = '<bearer_token>'
# access_token = '<access_token>'
# access_token_secret = '<access_token_secret>'

client = tweepy.Client(bearer_token=bearer_token)

# News Categories:
news_outlets = '((url:"www.bloomberg.com") OR (url:"www.fortune.com") OR (url:"www.theguardian.com") OR (url:"www.npr.org") OR (url:"www.salon.com") OR  (url:"www.newsweek.com") OR (url:"www.politico.com") OR (url:"www.pbs.org") OR (url:"www.economist.com") OR (url:"www.nbc.com") OR (url:"www.abcnews.com") OR (url:"www.cbsnews.com") OR (url:"www.cnn.com") OR (url:"www.foxnews.com") OR (url:"www.sfchronicle.com"))'

# documentation: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#build
query_params = {'query': news_outlets + ' -is:retweet',
           'tweet.fields': 'author_id,entities', 'max_results':100}

search_url = "https://api.twitter.com/2/tweets/search/recent"

def bearer_oauth(r):
    '''Construct an authentication header for the search service 
    '''
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r


def connect_to_endpoint(url, params):
    '''Send the Twitter API a request to get back some data on the query you built.
    '''
    # this is called a GET request
    response = requests.get(url, auth=bearer_oauth, params=params)
    
    if response.status_code != 200:  # code 200 indicates OK
        raise Exception(response.status_code, response.text)
    return response.json()

response = connect_to_endpoint(search_url, query_params)

In [ ]:
# Output an array given a search query:
def parse_response(res):
    if 'urls' in res['entities']:
        return ['www.twitter.com/anyuser/status/' + str(res['id']), res['text'], res['entities']['urls'][0]['expanded_url']]
    else:
        return ['www.twitter.com/anyuser/status/' + str(res['id']), res['text'], '']


# This is necessary because the API generates results in pages.
def paginate(r):
    token = r['meta']['next_token']
    new_params = query_params
    new_params.update({'pagination_token':token})
    return connect_to_endpoint(search_url, new_params)

responses = [response]

# We arbitrarily chose to use 10 pages of max length 100.
for i in range(10):
    responses.append(paginate(responses[i]))

# Save all the responses to a .csv file:
table = []
def add_to_table(res):
    for x in range(res['meta']['result_count']):
        row = []
        row += (parse_response(((res)['data'])[x]))
        table.append(row)

for r in responses:
    add_to_table(r)

df = pd.DataFrame({'Link': [i[0] for i in table], 
                   'Body': [i[1] for i in table],
                   'Article': [i[2] for i in table],
                   })
                   
df.to_csv('news_tweets.csv')

In [ ]:
for r in responses:
    print(r)
    break

In [ ]:
r['data']